# Processing public comments

### Formatting / setting up the data

In [1]:
%pip install tabula-py
%pip install jpype1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tabula
import pandas as pd

pdf_path = "Public comments - Northmet.pdf"
tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True, lattice=True)
tables_no_headers = [df.iloc[1:].copy() for df in tables if len(df) > 1]
df_all = pd.concat(tables_no_headers, ignore_index=True)

for col in df_all.select_dtypes(include='object').columns:
    df_all[col] = df_all[col].map(lambda x: str(x).replace('\r', ' ').replace('\n', ' ').strip() if pd.notnull(x) else x)

df_all.columns = [
    "Name of Sender",
    "Comment",
    "Issue",
    "Substantive / Non-Substantive",
    "Old / New",
    "Response ID",
    "RGU Consideration"
][:len(df_all.columns)]

rows_to_drop = []
for i in range(1, len(df_all)):
    name = df_all.iloc[i, 0]
    if pd.isna(name) or str(name).strip() == '':
        prev_comment = str(df_all.at[i - 1, "Comment"]).strip()
        curr_comment = str(df_all.at[i, "Comment"]).strip()
        df_all.at[i - 1, "Comment"] = prev_comment + ' ' + curr_comment
        rows_to_drop.append(i)

df_all.drop(index=rows_to_drop, inplace=True)
df_all.reset_index(drop=True, inplace=True)

df_all.to_csv("combined_cleaned_responses.csv", index=False)
display(df_all.head(20))

,Name of Sender,Comment,Issue,Substantive / Non-Substantive,Old / New,Response ID,RGU Consideration
0,Kathleen Whitson,PLEASE do NOT approve the mining. It will prof...,GEN,NS,X,1,NaN
1,Mark,The environment will eventually be polluted by...,FIN,NS,X,1,NaN
2,Bob Woodbury,Have there been other projects of this nature ...,PER,NS,X,1,NaN
3,Bob Woodbury,"I could go on in this vein, but my point is th...",PER,NS,X,1,NaN
4,Bob Woodbury,"We need to make a decision on what we know, no...",NEPA,NS,X,1,NaN
5,Bob Woodbury,This is not just a Northern Minnesota concern....,NEPA,NS,X,1,NaN
6,Bob Woodbury,What happens if the project is denied. There w...,ALT,NS,X,1,NaN
7,Bob Woodbury,Our lands are vast. This isn't the only place ...,ALT,NS,X,1,NaN
8,John-Marilyn Rossi,It is my understanding that returning the area...,ALT,S,O,8,"SDEIS Themes ALT 03, ALT 06, ALT 13"
9,John-Marilyn Rossi,Why is the state of Minnesota not requiring fu...,ALT,S,O,8,"SDEIS Themes ALT 03, ALT 06, ALT 13"


### Using BERTopic to process it

In [3]:
%pip uninstall keras --user
%pip install tf-keras --user
%pip install bertopic sentence-transformers umap-learn hdbscan --user

Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\matth\anaconda3\python.exe -m pip uninstall [options] <package> ...
  c:\Users\matth\anaconda3\python.exe -m pip uninstall [options] -r <requirements file> ...

no such option: --user


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt

comments = df_all.iloc[1:, 1].tolist()

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
topic_model = BERTopic(embedding_model=embedding_model, verbose=True)
topics, probs = topic_model.fit_transform(comments)
topic_model.get_topic_info()

ImportError: Traceback (most recent call last):
  File "C:\Users\matth\AppData\Roaming\Python\Python312\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.